In [1]:
import pandas as pd
from glob import glob
import numpy as np

In [2]:
all_subjects = []
all_dates = []
all_images = []
all_notes = []
# patient_groups = {}
for group in ["CN","MCI","AD"]:
    files = glob(f"Data/Pre-processed/{group}/*.nii.gz")
    'Data/Pre-processed/CN\\1000_sub-ADNI941S6546_ses-M048_T1w2_norm.nii.gz'
    def extract_sub(fp):
        sub = fp[fp.index("S"):]
        return sub[:sub[3:].index("_")+3].replace("_","")
    
    def extract_date(fp):
        if "_20" not in fp:
            all_notes.append(fp)
            return pd.NA
        all_notes.append(pd.NA)
        date = fp[fp.rfind("_20")+1:]
        return date[:date.index("_S")]

    all_subjects.extend([extract_sub(fp) for fp in files])
    all_dates.extend([extract_date(fp) for fp in files])

# all_images = np.stack(all_images)
# print(all_images.shape)

all_subjects = pd.DataFrame([all_subjects,all_dates,all_notes]).T
all_subjects.columns=["subject","date","note"]
all_subjects["date"] = pd.to_datetime(all_subjects["date"].str[:14],format="%Y%m%d%H%M%S")
all_subjects.reset_index(inplace=True)
all_subjects.rename(columns={"index":"imageid"},inplace=True)

all_subjects["Group"] = 0
all_subjects.loc[1326:1326+461,"Group"] = 1
all_subjects.loc[1326+461:1326+461+213,"Group"] = 2

all_subjects

,imageid,subject,date,note,Group
0,0,S6546,NaT,Data/Pre-processed/CN\1000_sub-ADNI941S6546_se...,0
1,1,S6570,NaT,Data/Pre-processed/CN\1001_sub-ADNI941S6570_se...,0
2,2,S6570,NaT,Data/Pre-processed/CN\1002_sub-ADNI941S6570_se...,0
3,3,S6574,NaT,Data/Pre-processed/CN\1003_sub-ADNI941S6574_se...,0
4,4,S6574,NaT,Data/Pre-processed/CN\1004_sub-ADNI941S6574_se...,0
...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,<NA>,2
1996,1996,S0916,2006-12-03 14:39:25,<NA>,2
1997,1997,S1262,2007-08-05 14:09:40,<NA>,2
1998,1998,S0404,2006-12-22 19:35:16,<NA>,2


In [3]:
suckers = all_subjects[pd.isna(all_subjects["date"])]
def extract_month(fp):
    m = fp[fp.index("ses-M")+4:]
    return m[:m.index("_")]
subs = suckers["subject"]
months = [extract_month(n) for n in suckers["note"]]

def open_prepare_df(name="All_Subjects_RECCMEDS_11Mar2025"):
    try:
        df = pd.read_csv(f"Data/Individual level/{name}.csv")
    except Exception as e:
        if "Archived" in name: raise e 
        return open_prepare_df("Archived/"+name)
    df["subject"] = df["PTID"].str.split("_")
    df["subject"] = df["subject"].str[1] + df["subject"].str[2]
    return df

df = open_prepare_df("All_Subjects_MRI3META_26Mar2025")[["subject","RID","VISCODE","VISCODE2","EXAMDATE"]]
dates = []
for i in range(len(months)):
    date = pd.NaT
    potential = df.loc[df["subject"]==subs[i]]
    checks = [months[i].lower()]
    checks += [checks[0].replace("m0","m")]
    if checks[0]=="m000": checks.append("sc")
    for c in checks:
        if (potential["VISCODE2"]==c).any():
            date = potential.loc[potential["VISCODE2"]==c, "EXAMDATE"].values[0]
            break
    if date is pd.NaT:
        date = potential["EXAMDATE"].values[-1] # Just grab one. This case happens 3 times
        # print(months[i])
        # print(potential)
    dates.append(date)

all_subjects.loc[pd.isna(all_subjects["date"]),"date"] = dates
all_subjects.rename(columns={"date":"imgDate"},inplace=True)
all_subjects.drop(columns="note",inplace=True)
all_subjects

,imageid,subject,imgDate,Group
0,0,S6546,2022-10-03 00:00:00,0
1,1,S6570,2018-08-31 00:00:00,0
2,2,S6570,2022-09-20 00:00:00,0
3,3,S6574,2018-09-07 00:00:00,0
4,4,S6574,2020-11-02 00:00:00,0
...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2
1996,1996,S0916,2006-12-03 14:39:25,2
1997,1997,S1262,2007-08-05 14:09:40,2
1998,1998,S0404,2006-12-22 19:35:16,2


In [4]:
adverse_events = open_prepare_df("All_Subjects_ADVERSE_11Mar2025")

A=set(adverse_events["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
adv_events = adverse_events[["subject","AERELAD","AERELCM","AERELFLRBTBN","AERELFLRBPR","AEHIMG","AERELTAU","AERELNAV","AERELMK","AERELPI","AEHLUMB","AERELCOVID","AERELPAN","AERELATESP","AEHCMEDS","AESERIOUS"]].fillna(0)
for col in adv_events:
    if col =="subject": continue
    col_ = adv_events[col]
    adv_events[col] = col_.astype("float")
    adv_events.loc[col_ > 0, col] = (5 - col_) / 4.
adv_events

1120 378 904 594


,subject,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,S2245,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,1.00,0.0,0.0,0.00,1.0,0.0
1,S6041,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.50,0.0,0.0,0.00,0.0,0.0
2,S6053,0.0,0.0,0.0,0.25,0.25,0.25,0.0,0.0,0.0,0.75,0.0,0.0,0.25,1.0,0.0
3,S6005,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,S2239,0.0,0.5,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,S7011,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
3421,S10031,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0
3422,S10031,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
3423,S6288,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0


In [5]:
num_events = all_subjects.groupby(by="subject").size().reset_index().rename(columns={0:"num_events"})
all_subjects = all_subjects.merge(num_events,how="left").fillna(0)
all_subjects = all_subjects.merge(adv_events.groupby(by="subject").sum().reset_index(),how="left").fillna(0)
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
backmeds = open_prepare_df("All_Subjects_BACKMEDS_11Mar2025")
backmeds = backmeds[["subject","KEYMED"]]
med_vocab = list(map(str,list(range(8))))
backmeds_ = []
names = ["No med",
"Aricept",
"Cognex",
"Exelon",
"Namenda",
"Razadyne",
"Anti-depressant",
"Other med"]
remap = {str(i):names[i] for i in range(len(names))}
for i, row in backmeds.iterrows():
    data = {"subject":row["subject"]}
    for word in med_vocab:
        data[remap[word]] = int(word in str(row["KEYMED"]))
    backmeds_.append(data)
backmeds_=pd.DataFrame(backmeds_)
# backmeds_["Num_meds"] = backmeds_.drop(columns=["subject","No med"]).sum(axis=1)
backmeds_["Num_visits"] = 1
backmeds_ = backmeds_.groupby(by="subject").sum().reset_index()
for col in backmeds_:
    if col in ["subject", "Num_visits"]: continue
    backmeds_[col] /= backmeds_["Num_visits"]

all_subjects=all_subjects.merge(backmeds_.groupby(by="subject").sum().reset_index(),how="left")
all_subjects.loc[all_subjects["Num_visits"]==0, "No med"] = 1
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,AESERIOUS,No med,Aricept,Cognex,Exelon,Namenda,Razadyne,Anti-depressant,Other med,Num_visits
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
neuropath = open_prepare_df("All_Subjects_NEUROPATH_11Mar2025")
neuropath = neuropath[["subject","NPWBRWT","NPGRCCA","NPGRLA","NPGRHA","NPGRSNH","NPGRLCH","NPAVAS","NPTAN","NPABAN","NPASAN","NPTDPAN","NPHISMB","NPHISG","NPHISSS","NPHIST","NPHISO","NPTHAL","NPBRAAK","NPNEUR","NPADNC","NPDIFF","NPAMY","NPINF","NPHEMO","NPOLD","NPARTER","NPWMR","NPPATH","NPNEC","NPPATHO","NPLBOD","NPNLOSS","NPHIPSCL","NPTDPA","NPTDPB","NPTDPC","NPTDPD","NPTDPE","NPFTDTAU","NPPICK","NPPDXS"]]

A=set(neuropath["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

# Due to the very limited amount of subjects who have been autopsied in our imaging data, we dont need to merge. not enough data.
# all_subjects=all_subjects.merge(neuropath,how="left")
# all_subjects

692 12 110 594


In [8]:
reccmed = pd.read_csv("Data/Individual level/Archived/All_Subjects_RECCMEDS_11Mar2025.csv", dtype="str")
reccmed["subject"] = reccmed["PTID"].str.split("_")
reccmed["subject"] = reccmed["subject"].str[1]+reccmed["subject"].str[2]

A=set(reccmed["subject"])
B=set(all_subjects["subject"])
# print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

reccmed = reccmed[["subject","CMMED","CMFREQNC","CMREASON", "CMBGN","CMEND","update_stamp"]]
reccmed["CMMED"] = reccmed["CMMED"].str.lower().str.replace("-"," ")
for suffix in [" formula", " oil"," sltb"," la"]:
    reccmed["CMMED"] = reccmed["CMMED"].str.removesuffix(suffix)
for overwrite in ["aspirin","oxybutynin","calcium","garlic","omega 3","probiotic","lactase","acetyl","vitamin b","vitamin c","ascorbic acid",
                  "tylenol","glucosamine","multivitamin","triamter","elidel","nephrovite","ammonium lactate","acetaminophen"]:
    mask = reccmed["CMMED"].str.contains(overwrite).fillna(0).astype("bool")
    reccmed.loc[mask, "CMMED"] = overwrite
# reccmed = reccmed[~pd.isna(reccmed["CMMED"])]
vocab = set(reccmed["CMMED"])
print(len(vocab))
vocab

3442


{'jamieson salmon',
 'rhodiola',
 'santura',
 'cranberry +c',
 'ultra flora plus',
 'senna',
 'bentyl',
 'refresh pm',
 'nystatin cream',
 'theophylline',
 'vitamins b',
 'dmg',
 'tobradex',
 'hydrochlorithiazide',
 'robitussin',
 'ocuvite with lutein',
 'bromocriptine',
 'vitamins b6 b12 e fa',
 'adalat',
 'cetiricine hcl',
 'citrical+vitamin d',
 'skelaxin',
 'epa',
 'insulin 7030',
 'losec',
 'zymar 0.3%',
 'prilosec',
 'clobetasol 0.05%',
 'renedil',
 'dicyclomine hcl',
 'triazolam',
 '"your eyes" supplement',
 'bactrim',
 'lamisil tablets',
 'hydroxyzine hcl',
 'patanol',
 'myacalcin',
 'docusate senna',
 'ferrous fumarate',
 'trentol',
 'osteo bi flex   triple strength',
 'dhea',
 'deep sea nasal spray',
 'nitrofurantoin',
 'cran actin',
 'hydrochlorathiazide',
 'osteo bi flex',
 'glucovance',
 'fluvastatin',
 'kenalog',
 'amoxicilin',
 'non acidic vitamin',
 'exelon parch',
 'captopril',
 'aerius',
 'enteric coated acetlysalicylic acid',
 'preser vision ( vit a,c,e)',
 'restasis

In [9]:
common_meds_vocab = reccmed["CMMED"].value_counts().head(30).keys().values

one_hot_df = pd.get_dummies(reccmed["CMMED"])[common_meds_vocab].astype("int")
one_hot_df.columns = ["med_"+col for col in one_hot_df.columns]
one_hot_df.rename(columns={"med_ 4":"med_unknown"},inplace=True)

reccmed_ = pd.concat([reccmed[["subject","CMREASON","CMFREQNC"]], one_hot_df], axis=1)
reccmed_

,subject,CMREASON,CMFREQNC,med_unknown,med_aspirin,med_multivitamin,med_calcium,med_aricept,med_namenda,med_fish,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,S0002,NaN,NaN,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72563,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72564,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72565,S10586,2,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72566,S10586,2,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
groups = reccmed_.drop(columns=["CMFREQNC","CMREASON"]).groupby(by="subject")
all_meds_hist = groups.mean()
all_meds_hist

,med_unknown,med_aspirin,med_multivitamin,med_calcium,med_aricept,med_namenda,med_fish,med_vitamin b,med_vitamin c,med_vitamin d,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
subject,,,,,,,,,,,,,,,,,,,,,
S0002,0.300000,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0003,0.111111,0.055556,0.055556,0.000000,0.000000,0.277778,0.000000,0.0,0.055556,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0004,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.0,0.043478,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0005,0.095238,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0
S0006,0.210526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S7122,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S7123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S7124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [11]:
all_subjects = all_subjects.merge(all_meds_hist.reset_index(),how="left")
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.04,0.0,0.04,0.0,0.000000,0.0,0.0,0.0
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.076923,0.0,0.0,0.0
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0


In [12]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("FAMHXSIB_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject","SIBDEMENT"]]
siblings["Num_siblings"] = 1
siblings = siblings.groupby(by="subject").sum()
siblings["SIBDEMENT"] /= siblings["Num_siblings"]
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
naive_dementia_rate = 0.001 # siblings["SIBDEMENT"].mean()
# all_subjects.loc[pd.isna(all_subjects["SIBDEMENT"]), "SIBDEMENT"] = naive_dementia_rate
# all_subjects = all_subjects.fillna(siblings["Num_siblings"].median()) # Num_siblings unknown set to zero
all_subjects

1836 402 1644 594


,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax,SIBDEMENT,Num_siblings
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,NaN,NaN
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,2.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,2.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,2.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,NaN,NaN
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,0.04,0.0,0.04,0.0,0.000000,0.0,0.0,0.0,NaN,NaN
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.076923,0.0,0.0,0.0,NaN,NaN
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,NaN,NaN


In [13]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("FAMHXPAR_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject","MOTHDEM","FATHDEM","MOTHAGE","FATHAGE"]]
siblings = siblings.groupby(by="subject").mean()
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects["MOTHDEM"] = all_subjects["MOTHDEM"]
all_subjects["FATHDEM"] = all_subjects["FATHDEM"]
all_subjects

1914 421 1741 594


,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,med_omega 3,med_metformin,med_prednisone,med_flomax,SIBDEMENT,Num_siblings,MOTHDEM,FATHDEM,MOTHAGE,FATHAGE
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,NaN,NaN,1.0,0.0,94.0,69.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2.0,0.0,1.0,81.0,82.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2.0,0.0,1.0,81.0,82.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.0,94.0,52.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.0,94.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,0.076923,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("PHYSICAL_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("PX")]]
siblings = siblings.groupby(by="subject").mean()

all_subjects = all_subjects.merge(siblings,on="subject",how="left")
# all_subjects = all_subjects.fillna(1) # In case we have missing values, 1 represents a "normal" case.
all_subjects

4043 594 4043 594


,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,PXHEART,PXABDOM,PXEXTREM,PXEDEMA,PXPERIPH,PXSKIN,PXMUSCUL,PXBACK,PXOTHER,PXABNORM
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,1.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,1.5,1.0,1.0,1.0,1.0,1.5,1.5,1.0,1.0,1.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,1.5,1.0,1.0,1.0,1.0,1.5,1.5,1.0,1.0,1.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,1.5,2.0,1.5,1.0,1.0,1.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,1.5,2.0,1.5,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,2.0,2.0,1.0,NaN,1.0,1.0
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,1.0,2.0,2.0,NaN,1.0,1.0


In [15]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("PTDEMOG_21Mar2025")
exclude = ["PTID","PTDOB","PTASIAN","PTADBEG","PTIDENT","PTORIENT","PTENGSPK","PTETHCATH","PTWORK","PTWORKHS","PTOPI","PTBIRPL","PTIMMAGE","PTIMMWHY","PTORIENTOT","PTENGSPKAGE","PTSPTIM","PTSPOTTIM","PTBORN","PTBIRPR","PTBIRGR"]
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("PT")and c not in exclude and "LANG" not in c]]
# siblings = siblings.astype("float")
siblings["PTRACCAT"] = siblings["PTRACCAT"].str.split("|").str[0].astype("float")
siblings = siblings.groupby(by="subject").first().replace(-4,pd.NA)# In case we have missing values, 1 represents a "normal" case.
# siblings["PTHAND"].unique()
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTRTYR,PTHOME,PTCOGBEG,PTADDX,PTETHCAT,PTRACCAT
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,1.0,2.0,20.0,1.0,2005.0,1.0,9999.0,9999.0,2.0,5.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,18.0,0.0,NaN,1.0,9999.0,9999.0,2.0,5.0
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,18.0,0.0,NaN,1.0,9999.0,9999.0,2.0,5.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2007.0,1.0,9999.0,9999.0,2.0,5.0
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2007.0,1.0,9999.0,9999.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2005.0,1.0,NaN,NaN,2.0,5.0
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,1984.0,2.0,NaN,NaN,2.0,5.0
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,1.0,3.0,12.0,1.0,2005.0,1.0,NaN,NaN,2.0,5.0
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,1.0,1.0,14.0,0.0,NaN,6.0,NaN,NaN,2.0,5.0


In [16]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("VITALS_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
exclude = ["VSHGTSC","VSTMPSRC"]
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("VS")and c not in exclude]]
siblings.loc[siblings["VSWTUNIT"]==1, "VSWEIGHT"] *= 0.45359237 # pounds
siblings.loc[siblings["VSHTUNIT"]==1, "VSHEIGHT"] *= 2.54 # inches
siblings.loc[siblings["VSTMPUNT"]==1, "VSTEMP"] = (siblings.loc[siblings["VSTMPUNT"]==1, "VSTEMP"] - 32) / 1.8 # fahrenheight
siblings = siblings.drop(columns=[c for c in siblings.columns if "UN" in c])
siblings = siblings.groupby(by="subject").mean()
# siblings
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects

4140 594 4140 594


,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,PTADDX,PTETHCAT,PTRACCAT,VSWEIGHT,VSHEIGHT,VSBPSYS,VSBPDIA,VSPULSE,VSRESP,VSTEMP
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,9999.0,2.0,5.0,76.611751,167.000000,137.750000,70.250000,68.75,18.250000,36.819444
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,9999.0,2.0,5.0,90.869671,172.000000,149.000000,69.666667,58.00,18.666667,25.062963
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,9999.0,2.0,5.0,90.869671,172.000000,149.000000,69.666667,58.00,18.666667,25.062963
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,9999.0,2.0,5.0,58.096111,158.000000,114.200000,64.800000,70.40,17.600000,29.855556
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,9999.0,2.0,5.0,58.096111,158.000000,114.200000,64.800000,70.40,17.600000,29.855556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,NaN,2.0,5.0,84.096025,32.360000,135.400000,84.400000,66.80,17.200000,36.233333
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,NaN,2.0,5.0,71.050709,31.852000,158.600000,75.600000,51.80,19.600000,35.566667
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,NaN,2.0,5.0,50.711627,23.336667,135.666667,67.500000,69.00,17.666667,36.333333
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,NaN,2.0,5.0,65.625744,29.566000,127.800000,67.800000,68.40,19.200000,36.120000


In [17]:
siblings = open_prepare_df("All_Subjects_APOERES_24Mar2025")
siblings = siblings[["subject","GENOTYPE"]].rename(columns={"GENOTYPE":"APOEGEN_1"})
siblings["APOEGEN_2"] = siblings["APOEGEN_1"].str.split("/").str[1].astype("int")
siblings["APOEGEN_1"] = siblings["APOEGEN_1"].str.split("/").str[0].astype("int")
# siblings=siblings.groupby(by="subject").first().reset_index()
all_subjects = all_subjects.merge(siblings, how="left", on="subject")

In [18]:
siblings = open_prepare_df("All_Subjects_FCI_24Mar2025")
siblings = siblings[["subject"]+[c for c in siblings.columns if c.endswith("TIME") or "SCORE" in c]]
siblings=siblings.dropna().reset_index(drop=True)
siblings=siblings.groupby(by="subject").mean().reset_index()
# siblings = siblings[["subject","GENOTYPE"]].rename(columns={"GENOTYPE":"APOEGEN"})
all_subjects=all_subjects.merge(siblings, how="left", on="subject")

In [19]:
all_subjects[["subject","imgDate"]]

,subject,imgDate
0,S6546,2022-10-03 00:00:00
1,S6570,2018-08-31 00:00:00
2,S6570,2022-09-20 00:00:00
3,S6574,2018-09-07 00:00:00
4,S6574,2020-11-02 00:00:00
...,...,...
1995,S0300,2008-10-13 16:43:20
1996,S0916,2006-12-03 14:39:25
1997,S1262,2007-08-05 14:09:40
1998,S0404,2006-12-22 19:35:16


In [20]:
siblings = open_prepare_df("All_Subjects_MMSE_24Mar2025")

B=siblings[["subject","VISDATE","MMSCORE"]]
A=all_subjects[["subject","imgDate"]]

# Ensure dates are datetime format
A["imgDate"] = pd.to_datetime(A["imgDate"])
B["VISDATE"] = pd.to_datetime(B["VISDATE"])

# Sort B by subject and VISDATE for correct interpolation
A = A.sort_values(["subject", "imgDate"])
all_subjects["MMSE_INTERP"] = None
for subject, df_A in A.groupby("subject"):
    df_B = B[B["subject"] == subject]

    if not df_B.empty:
        # Interpolate using np.interp (requires numeric timestamps)
        img_dates_num = df_A["imgDate"].astype(np.int64)  # Convert to numeric
        vis_dates_num = df_B["VISDATE"].astype(np.int64)
        mm_scores = df_B["MMSCORE"]

        if len(vis_dates_num) > 1:
            interp_values = np.interp(img_dates_num, vis_dates_num, mm_scores)
        else:
            interp_values = [mm_scores.values[0]] * len(img_dates_num)  # Single value case

        # Assign interpolated values back
        all_subjects.loc[df_A.index, "MMSE_INTERP"] = interp_values

all_subjects


C:\Users\PieHu\AppData\Local\Temp\ipykernel_5868\810384567.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A["imgDate"] = pd.to_datetime(A["imgDate"])
C:\Users\PieHu\AppData\Local\Temp\ipykernel_5868\810384567.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B["VISDATE"] = pd.to_datetime(B["VISDATE"])


,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,CHKCOMPOSTIME,CHKCOMPOSSCORE,BANKSTATSCORE,TOTCOMPTIME,FCISCORE,MEDINSTIME,QUES5TIME,SNGLCHKTIME,COMPLXCHKTIME,MMSE_INTERP
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,251.000000,42.000000,12.333333,265.666667,65.000000,11.333333,3.333333,104.0,147.000000,28.0
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,204.000000,47.000000,14.000000,206.000000,73.000000,1.000000,1.000000,80.0,124.000000,29.990591
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,204.000000,47.000000,14.000000,206.000000,73.000000,1.000000,1.000000,80.0,124.000000,28.0
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,254.666667,46.666667,12.666667,273.333333,70.666667,13.666667,5.000000,84.0,170.666667,28.027778
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,254.666667,46.666667,12.666667,273.333333,70.666667,13.666667,5.000000,84.0,170.666667,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.784265
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.6247


In [21]:
siblings = open_prepare_df("All_Subjects_MMSE_24Mar2025")
siblings = siblings[["subject","MMSCORE"]].rename(columns={"MMSCORE":"MMSE_MEAN"})
siblings=siblings.dropna().reset_index(drop=True)
siblings=siblings.groupby(by="subject").mean().reset_index()
# siblings
# siblings = siblings[["subject","GENOTYPE"]].rename(columns={"GENOTYPE":"APOEGEN"})
all_subjects=all_subjects.merge(siblings, how="left", on="subject")
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,CHKCOMPOSSCORE,BANKSTATSCORE,TOTCOMPTIME,FCISCORE,MEDINSTIME,QUES5TIME,SNGLCHKTIME,COMPLXCHKTIME,MMSE_INTERP,MMSE_MEAN
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,42.000000,12.333333,265.666667,65.000000,11.333333,3.333333,104.0,147.000000,28.0,27.666667
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,47.000000,14.000000,206.000000,73.000000,1.000000,1.000000,80.0,124.000000,29.990591,29.000000
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,47.000000,14.000000,206.000000,73.000000,1.000000,1.000000,80.0,124.000000,28.0,29.000000
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,46.666667,12.666667,273.333333,70.666667,13.666667,5.000000,84.0,170.666667,28.027778,29.000000
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,46.666667,12.666667,273.333333,70.666667,13.666667,5.000000,84.0,170.666667,30.0,29.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,25.000000
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,18.000000
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.784265,12.600000
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.6247,20.250000


In [22]:
siblings = open_prepare_df("All_Subjects_MMSE_24Mar2025")
all_subjects["first_mmse_year"]=pd.to_datetime(siblings[["subject","USERDATE"]].groupby(by="subject").first().reset_index()["USERDATE"]).dt.year

In [23]:
brain_volume = pd.read_csv("Data/Individual level/brain_volume.csv").rename(columns={"Unnamed: 0":"imageid"}).drop(columns="subject")
all_subjects=all_subjects.merge(brain_volume, how="left", on="imageid")
# all_subjects

In [24]:
all_subjects.to_csv("Data/subjects_nan.csv")
# all_subjects = pd.read_csv("Data/subjects.csv").drop(columns="Unnamed: 0")
all_subjects

,imageid,subject,imgDate,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,...,FCISCORE,MEDINSTIME,QUES5TIME,SNGLCHKTIME,COMPLXCHKTIME,MMSE_INTERP,MMSE_MEAN,first_mmse_year,concave,convex
0,0,S6546,2022-10-03 00:00:00,0,4,0.0,0.25,0.0,0.0,0.0,...,65.000000,11.333333,3.333333,104.0,147.000000,28.0,27.666667,2005,1631980.0,1.770903e+06
1,1,S6570,2018-08-31 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,73.000000,1.000000,1.000000,80.0,124.000000,29.990591,29.000000,2005,1628163.0,1.767325e+06
2,2,S6570,2022-09-20 00:00:00,0,3,0.0,0.00,0.0,0.0,0.0,...,73.000000,1.000000,1.000000,80.0,124.000000,28.0,29.000000,2005,1580466.0,1.704634e+06
3,3,S6574,2018-09-07 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,70.666667,13.666667,5.000000,84.0,170.666667,28.027778,29.000000,2005,1477454.0,1.629739e+06
4,4,S6574,2020-11-02 00:00:00,0,5,0.0,0.00,0.0,0.0,0.0,...,70.666667,13.666667,5.000000,84.0,170.666667,30.0,29.000000,2005,1595403.0,1.739119e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2008-10-13 16:43:20,2,12,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,26.0,25.000000,2011,1654017.0,1.797397e+06
1996,1996,S0916,2006-12-03 14:39:25,2,18,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,21.0,18.000000,2011,1597718.0,1.738169e+06
1997,1997,S1262,2007-08-05 14:09:40,2,14,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,15.784265,12.600000,2010,1601268.0,1.727674e+06
1998,1998,S0404,2006-12-22 19:35:16,2,11,0.0,0.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,19.6247,20.250000,2011,1609039.0,1.749592e+06
